In [1]:
'''
- read java csv
- remove source column
- add negative sample by shuffling them
- split into train, test and validation
- save paths as text files
-ready to feed the model!
'''
import pandas as pd
import numpy as np
import random as rd
from sklearn.utils import shuffle
import random
random.seed(10)
def get_index():
    #random generation explanation 
    #https://www.sciencedirect.com/topics/computer-science/mersenne-twister
    #first fragment position
    rr1, rc1 = rd.randint(0,len(data)-1), rd.randint(0,1)
    #second fragment position
    rr2, rc2 = rd.randint(0,len(data)-1), rd.randint(0,1)
    
    # position of ragments is order invariant,the only constraint, it can not be on same row
    #check if in same row
    while(rr1==rr2):
        rr2, rc2 = rd.randint(0,len(data)-1), rd.randint(0,1)
    
    #column position for first fragment
    rc = rd.choice([rc1,rc2])
    first_pos = (rr1, rc)
    second_pos = (rr2, rc1+rc2-rc)
    
    return (first_pos, second_pos)

def generate_neg_pairs_index(neg_pairs):
    '''
    returns random index (row, col) for sampling negative data
    '''
    if len(neg_pairs) == 1000:
        return
    
    pair = get_index()
    if pair in neg_pairs:
        #print('Already taken this pair! Generating another pair...')
        generate_neg_pairs_index(neg_pairs)
    else:
        neg_pairs.append(pair)
        
    return pair

def get_complete_dataset(data):
    
    #generate negative pairs
    neg_pairs = []
    c = 0
    while(c<1000):
        generate_neg_pairs_index(neg_pairs)
        c = len(neg_pairs) #neg_paairs is incremented only when a pair is added

    #copy all positives to a new csv
    dataset = data.copy()
    
    #add negative pairs info to dataset
    for pair in neg_pairs:
        #print(pair)
        #code fragments
        cf1 = pair[0]
        cf2 = pair[1]
        
        #row to add
        row = pd.DataFrame(
            {
            'fragment_1': [data.iloc[cf1[0]][cf1[1]]],
            'fragment_2': [data.iloc[cf2[0]][cf2[1]]], 
            'label': [-1]
            }
        )
        #append the row
        dataset = pd.concat([dataset, row])
    
    return dataset

In [2]:
#main
#read and kep only relevant columns
path ='./dataset_splitted/_java_dataset_all_positive.csv'
keep =  ['first_fragment', 'second_fragment', 'label']
data = pd.read_csv(path, usecols = keep )
#rename columns
data.rename(columns = {'first_fragment':'fragment_1', 'second_fragment':'fragment_2'}, inplace = True)
print('Number of positive pairs: ', data.shape[0])

#generate negative samples and get complete dataset
dataset = get_complete_dataset(data)
print('Number of positive pairs: ', dataset.shape[0])

Number of positive pairs:  1000
Number of positive pairs:  2000


In [3]:
dataset.to_csv('./dataset_diff/datasetv1.csv')

In [4]:
dataset

,fragment_1,fragment_2,label
0,./dataset_splitted/java/98501.txt,./dataset_splitted/java/98502.txt,1
1,./dataset_splitted/java/74901.txt,./dataset_splitted/java/74902.txt,1
2,./dataset_splitted/java/99101.txt,./dataset_splitted/java/99102.txt,1
3,./dataset_splitted/java/76101.txt,./dataset_splitted/java/76102.txt,1
4,./dataset_splitted/java/77501.txt,./dataset_splitted/java/77502.txt,1
...,...,...,...
0,./dataset_splitted/java/55301.txt,./dataset_splitted/java/30402.txt,-1
0,./dataset_splitted/java/31901.txt,./dataset_splitted/java/24901.txt,-1
0,./dataset_splitted/java/80902.txt,./dataset_splitted/java/65501.txt,-1
0,./dataset_splitted/java/11201.txt,./dataset_splitted/java/13402.txt,-1


In [5]:
import matplotlib.pyplot as plt
dataset['fragment_1'].value_counts()

./dataset_splitted/java/5101.txt     5
./dataset_splitted/java/34401.txt    5
./dataset_splitted/java/50401.txt    4
./dataset_splitted/java/23801.txt    4
./dataset_splitted/java/84601.txt    4
                                    ..
./dataset_splitted/java/52301.txt    1
./dataset_splitted/java/25101.txt    1
./dataset_splitted/java/27901.txt    1
./dataset_splitted/java/90601.txt    1
./dataset_splitted/java/2502.txt     1
Name: fragment_1, Length: 1412, dtype: int64

In [6]:
import pandas as pd
df = pd.read_csv('./dataset_splitted/_java_dataset_all_positive.csv')
df.columns

Index(['source', 'first_fragment', 'second_fragment', 'question_id',
       'first_fragment_answer_id', 'second_fragment_answer_id', 'label'],
      dtype='object')

In [7]:
p= df.question_id.value_counts().sort_values().tail(20)#.plot(kind='bar', ylabel = 'Frequency', xlabel='Questions ID', color=(0.2, 0.4, 0.6, 0.6))

In [8]:
p = p.to_frame()

In [9]:
p['question_id']

1519736      7
13692221     7
25922372     7
34342816     8
12683533     9
35531747     9
2044033     10
363681      10
15045640    11
10647520    11
32447043    12
5357455     13
19552754    13
6309407     14
13119926    18
1248510     20
11544843    20
7230819     35
3911966     42
19459197    53
Name: question_id, dtype: int64

In [10]:
pd.read_csv('./dataset_diff/datasetv1.csv')

,Unnamed: 0,fragment_1,fragment_2,label
0,0,./dataset_splitted/java/98501.txt,./dataset_splitted/java/98502.txt,1
1,1,./dataset_splitted/java/74901.txt,./dataset_splitted/java/74902.txt,1
2,2,./dataset_splitted/java/99101.txt,./dataset_splitted/java/99102.txt,1
3,3,./dataset_splitted/java/76101.txt,./dataset_splitted/java/76102.txt,1
4,4,./dataset_splitted/java/77501.txt,./dataset_splitted/java/77502.txt,1
...,...,...,...,...
1995,0,./dataset_splitted/java/55301.txt,./dataset_splitted/java/30402.txt,-1
1996,0,./dataset_splitted/java/31901.txt,./dataset_splitted/java/24901.txt,-1
1997,0,./dataset_splitted/java/80902.txt,./dataset_splitted/java/65501.txt,-1
1998,0,./dataset_splitted/java/11201.txt,./dataset_splitted/java/13402.txt,-1
